# Cython
Cython is a Python compiler that makes writing C extensions for Python as easy as Python itself. Cython translates Python code to C/C++ code but additionally supports calling C functions and declaring C types on variables and class attributes. This allows the compiler to generate very efficient C code from Cython code.

This makes Cython the ideal language for wrapping external C libraries, and for fast C modules that speed up the execution of Python code.

## Installation
If you use `pip`, you can install Cython through
```bash
pip install cython
```
or if Anaconda is your package manager, you can use
```bash
conda install cython
```
## Usage
In Jupyter notebook, you can load the Cython extension as follows:

In [ ]:
%load_ext Cython

Then, prefix a cell with the `%%cython` marker to compile it

In [ ]:
%%cython

a: cython.int = 0
for i in range(int(1e7)):
    a += i
print(a)


Using the `--annotate` option will show Cython’s code analysis.

In [ ]:
%%cython --annotate

a: cython.int = 0
for i in range(int(1e7)):
    a += i
print(a)

Of course, we can test performance of compiled and native Python.

In [ ]:
%%cython

def sum_numbers_cython(int n):
    #This code includes type declarations (e.g., cdef int) for variables,
    #which allows Cython to generate more efficient C code.
    cdef int result = 0
    cdef int i
    for i in range(1, n+1):
        result += i
    return result

In [ ]:
def sum_numbers(n):
    result = 0
    for i in range(1, n+1):
        result += i
    return result

In [ ]:
N = 2000
%timeit sum_numbers(N)
%timeit sum_numbers_cython(N)

## Cythonize outside Jupyter
`cythonize` is a function provided by the `Cython` library that is used to compile `Cython` code (.pyx files) into `C` code and then compile the `C` code into a `Python` extension module. In short, it is the process of converting `Cython` code into a form that can be used in `Python` and typically results in improved performance for computationally intensive tasks. cythonize simplifies the build process for `Cython` code, making it more accessible for `Python` developers who want to optimize their code.

Consider the following example, found in `examples/`

In [ ]:
from os import system

In [ ]:
system("cat examples/module1.pyx")

The following script will compile the `cython` code

In [ ]:
system("cat examples/setup.py")

We therefore execute it to compile `module1.pyx` using the following `bash` expression
```bash
    python setup.py build_ext --inplace
```
This command compiles your `Cython` code into a shared library or a `Python` extension module, and the `--inplace` flag tells `Python` to put the output files in the current directory.

In [ ]:
system("cd examples/ && python setup.py build_ext --inplace")

With this, we can already use the `Cython` module in `Python`. For example, in a `Python` script or Jupyter Notebook cell, we can write

In [ ]:
system("cat examples/main.py")

In [ ]:
system("python examples/main.py")

## Functions and libraries
### Numpy

Consider the following piece of code, where we define a function that calculates the aritmethic mean of a given array.

In [ ]:
%%cython

# To define a function we use the "cpdef" instruction
cpdef double cython_mean(long[:] array):
    cdef int n = array.shape[0]
    cdef double sum_val = 0.
    cdef int ii
    for ii in range(n):
        sum_val += array[ii]

    return sum_val / n

and now we test this function with numpy arrays:

In [ ]:
import numpy as np

In [ ]:
arr = np.random.randint(1,100,int(1e8), dtype=np.int64)
result = cython_mean(arr)
print(result)

In [ ]:
# Let us test times
%timeit cython_mean(arr)
%timeit np.mean(arr)

Now consider the following code, which aims to perform matrix multiplication using `Cython`

In [ ]:
%%cython

import numpy as np
cimport numpy as np

cpdef np.ndarray[double, ndim=2] matrix_multiply(np.ndarray[double, ndim=2] A, np.ndarray[double, ndim=2] B):
    cdef int m, n, p
    m = A.shape[0]
    n = A.shape[1]
    p = B.shape[1]
    
    cdef np.ndarray[double, ndim=2] result = np.zeros((m, p), dtype=np.float64)
    
    cdef int i, j, k
    for i in range(m):
        for j in range(p):
            for k in range(n):
                result[i, j] += A[i, k] * B[k, j]
    
    return result

In [ ]:
# Create two NumPy arrays
A = np.random.rand(1000, 1000)
B = np.random.rand(1000, 1000)

# Call the Cython function for matrix multiplication
result = matrix_multiply(A, B)

# Print the result
print(result)

In [ ]:
%timeit matrix_multiply(A, B)
%timeit A.dot(B)

### Cython
Consider the following `Cython` code, used to calculate the CDF of a normal distribution.

In [ ]:
%%cython
cimport numpy as np
from scipy.stats import norm

cpdef np.ndarray[double] cdf_normal(np.ndarray[double] x, double loc, double scale):
    return norm.cdf(x, loc, scale)

In [ ]:
import numpy as np
from scipy.stats import norm

# Generate test data
x = np.random.normal(0, 1, int(1e7))

# Calculate CDF using SciPy
%timeit norm.cdf(x, loc=0, scale=1)

# Calculate CDF using the Cython-optimized function
%timeit cdf_normal(x, loc=0, scale=1)

### Sympy

Let's do a bit of symbolic differentiation.

In [ ]:
%%cython

from sympy import symbols, diff

cpdef symbolic_differentiation():
    x = symbols('x')
    expression = x**3 + 2*x**2 + 3*x + 4
    derivative = diff(expression, x)
    return derivative

In [ ]:
x = symbols('x')
expression = x**3 + 2*x**2 + 3*x + 4

%timeit diff(expression, x)
%timeit symbolic_differentiation()